Given the time series of summer coherence extract the glacier outline

In [1]:
import glob
import os

import rasterio
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import numpy as np

In [2]:
coh_max_filenames = [
    '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T117/coh_max.tif',
    '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T15/coh_max.tif',
    '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T168/coh_max.tif',
    '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T44/coh_max.tif',
    '/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/sar_coherence_T95/coh_max.tif'
]
output_folder = "/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/coh_timseries/mosaic"
mask_filename = "/mnt/CEPH_PROJECTS/ESA_ClouDInSAR/Case_studies/InSAR_coherence_glaciers/glacier_inventories/GI_ST_1997/SGIhom_1997_v2025.shp"

In [4]:
coh_max_stack_filename = os.path.join(output_folder, 'coh_max.vrt')
cmd = f'gdalbuildvrt -separate {coh_max_stack_filename} {" ".join(coh_max_filenames)}'
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

Create a binary classification map combining all the orbits

In [6]:
with rasterio.open(coh_max_stack_filename) as src:
    
    coh_max = src.read()
    coh_th = np.sum(coh_max < 0.5, axis=0) > 0

    with rasterio.open(
        os.path.join(output_folder, 'coh_th.tif'),
        'w',
        driver='GTiff',
        height=coh_th.shape[0],
        width=coh_th.shape[1],
        count=1,
        dtype=np.uint8,
        crs=src.crs,
        transform=src.transform
    ) as dst:
        dst.write(coh_th, 1)

Mask the result using the glacier inventory 1997

In [9]:
shp = gpd.read_file(mask_filename)
geoms = shp.geometry.values  # list of shapely geometries
geometries = [mapping(geom) for geom in shp.geometry if geom is not None]

with rasterio.open(os.path.join(output_folder, 'coh_th.tif')) as src:
    
    out_image, out_transform = mask(
        src,
        shapes=geometries,
        crop=True,
    )
    
    out_meta = src.meta.copy()
    out_meta.update({
        "height": out_image.shape[1],
        "width":  out_image.shape[2],
        "transform": out_transform
    })

with rasterio.open(os.path.join(output_folder, 'coh_th_masked.tif'), "w", **out_meta) as dst:
    dst.write(out_image)